# Parser Gold Mapping

In [150]:
import json
import numpy as np
import conll16st.aligner as aligner
import conll16st.partial_scorer as ps
import read_write_files as rw

In [181]:
gold_path = "data/gold_standard/blind/gold.json"
gold_list = rw.read_json(gold_path)

In [182]:
predicted_lists = []

In [183]:
parse_path = "data/submissions/randomized/oslopots.json"
predicted_list = rw.read_json(parse_path)
predicted_lists += [("oslopots",predicted_list)]

In [184]:
parse_path = "data/submissions/randomized/nguyenlab.json"
predicted_list = rw.read_json(parse_path)
predicted_lists += [("nguyenlab",predicted_list)]

In [185]:
parse_path = "data/submissions/randomized/steven.json"
predicted_list = rw.read_json(parse_path)
predicted_lists += [("steven",predicted_list)]


In [186]:
def change_sets_to_list(alignments):
    for gold,pred in alignments:
        if gold != None:
            gold["Arg1"]["TokenIndexSet"] = list(gold["Arg1"]["TokenIndexSet"])
            gold["Arg2"]["TokenIndexSet"] = list(gold["Arg2"]["TokenIndexSet"])
        if pred != None:
            pred["Arg1"]["TokenIndexSet"] = list(pred["Arg1"]["TokenIndexSet"])
            pred["Arg2"]["TokenIndexSet"] = list(pred["Arg2"]["TokenIndexSet"])
        
    return alignments

In [187]:
def align_parsers_to_gold(gold_rel,parsers):
    total_alignment = {gold["ID"]:{"gold":gold,"parsers":[]} for gold in gold_rel}
    parsers_not_mappable = []
    
    parser_names = [name for name,parser in parsers]
    
    for name,parser_relations in parsers:
        print(parser_relations)
        arg1_alignment, arg2_alignment, relation_alignment = aligner.align_relations(
            gold_rel, 
            parser_relations, 
            0.7)
         
        relation_alignment = change_sets_to_list(relation_alignment)
        for gold_align,pred_align in relation_alignment:
            if gold_align == None:
                parsers_not_mappable += [pred_align]
            else:
                total_alignment[gold_align["ID"]]["parsers"] += [pred_align]
                total_alignment[gold_align["ID"]]["parser_names"] = parser_names
     
    return total_alignment,parsers_not_mappable
        

In [180]:
print(predicted_lists)

[('oslopots', 'data/submissions/randomized/oslopots.json'), ('nguyenlab', 'data/submissions/randomized/nguyenlab.json'), ('steven', 'data/submissions/randomized/steven.json'), ('oslopots', [{u'DocID': u'wikinews_128079', u'Arg1': {u'TokenList': [300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335]}, u'Arg2': {u'TokenList': [337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356]}, u'Parser': u'oslopots', u'Connective': {u'RawText': u'', u'TokenList': [337, 338]}, u'Sense': [u'EntRel'], u'Type': u'AltLex', u'ID': 215, u'Change': u'Arg2'}, {u'DocID': u'wikinews_82895', u'Arg1': {u'TokenList': [323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339]}, u'Arg2': {u'TokenList': [341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359]}, u'Parser': u'oslopots

In [179]:
total_alignment,not_mappables = align_parsers_to_gold(
    gold_rel=gold_list,
    parsers=predicted_lists)

data/submissions/randomized/oslopots.json


TypeError: string indices must be integers, not str

In [149]:
save_json(total_alignment.values(),"data/project_files/total_alignment.json")
save_json(not_mappables,"data/project_files/not_mappable.json")